In [151]:
from sklearn.datasets import load_iris, load_diabetes
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')
% matplotlib inline



- `self.data` contains entire data frame
- `self.independent` contains features
- `self.target` contains the label column name

In [152]:
class DecisionTree:

    def fit(self, data, target):
        self.data = data
        self.target = target
        self.independent = self.data.columns.tolist()
        self.independent.remove(target)
    
    def predict(self, data):
        return np.array([self.__flow_data_thru_tree(row) for row in data.values])

    def __flow_data_thru_tree(self, row):
        return self.data[self.target].value_counts() \
                   .apply(lambda x: x/len(self.data)).tolist()

In [153]:
train = pd.read_csv('DT_train_preprocessed.csv')
test = pd.read_csv('DT_test_preprocessed.csv')
train.head(3)

,Age,Fare,Embarked_C,Embarked_Q,Embarked_S,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,...,Pclass_1,Pclass_2,Pclass_3,Master,Miss,Mr,Mrs,Officer,Royalty,Survived
0,22.0,7.2500,0,0,1,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0.0
1,38.0,71.2833,1,0,0,0,0,1,0,0,...,1,0,0,0,0,0,1,0,0,1.0
2,26.0,7.9250,0,0,1,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,1.0


Currently we are just returning mean of all predictions

In [154]:
model = DecisionTree()
model.fit(train, 'Survived')
predictions = model.predict(test)[0:5]
predictions

array([[0.61616162, 0.38383838],
       [0.61616162, 0.38383838],
       [0.61616162, 0.38383838],
       [0.61616162, 0.38383838],
       [0.61616162, 0.38383838]])

Calculate impurity at a node level. 

In [155]:
def __calculate_impurity_score(self, data):
    if data is None or data.empty: return 0
    probs = np.array(train.Survived.value_counts(normalize=True))
    return probs.dot(1-probs)

Take one column , find the best split based on information gain over various values 

In [156]:
def __find_best_split_for_column(self, col):
    x = self.data[col]
    unique_values = x.unique()
    if len(unique_values) == 1: return None, None
    information_gain = None
    split = None
    # Iterate through all possible values
    for val in unique_values:
        left = x <= val
        right = x > val
        left_data = self.data[left]
        right_data = self.data[right]
        left_impurity = self.__calculate_impurity_score(left_data[self.target])
        right_impurity = self.__calculate_impurity_score(right_data[self.target])
        score = self.__calculate_information_gain(left_count = len(left_data),
                                                  left_impurity = left_impurity,
                                                  right_count = len(right_data),
                                                  right_impurity = right_impurity)
        # Store the best split
        if information_gain is None or score > information_gain: 
            information_gain = score 
            split = val
    return information_gain, split

def __calculate_information_gain(self, left_count, left_impurity, right_count, right_impurity):
    # weighted impurity score on nodes
    return self.impurity_score - ((left_count/len(self.data)) * left_impurity + \
                                  (right_count/len(self.data)) * right_impurity)

Find best split across all independent variables and store it 

In [157]:
def __find_best_split(self):
    best_split = {}
    for col in self.independent:
        information_gain, split = self.__find_best_split_for_column(col)
        if split is None: continue
        if not best_split or best_split["information_gain"] < information_gain:
            best_split = {"split": split, 
                          "col": col, 
                          "information_gain": information_gain}
    return best_split["split"], best_split["col"]

Create multiple branches with recursion

In [158]:
def __create_branches(self):
    self.left = DecisionTree()
    self.right = DecisionTree()
    left_rows = self.data[self.data[self.split_feature] <= self.criteria] 
    right_rows = self.data[self.data[self.split_feature] > self.criteria] 
    self.left.fit(data = left_rows, target = self.target)
    self.right.fit(data = right_rows, target = self.target)

we add max depth attribute to limit the length of the tree

In [159]:
def __create_branches(self):
    self.left = DecisionTree(max_depth = self.max_depth, 
                             depth = self.depth + 1)
    self.right = DecisionTree(max_depth = self.max_depth, 
                             depth = self.depth + 1)

Prediction

We are setting 2 properties that would check for leaf node and probability at a node

In [160]:
@property
def is_leaf_node(self): return self.left is None

@property
def probability(self): 
    return self.data[self.target].value_counts().apply(lambda x: x/len(self.data)).tolist()

In this case, the @property decorator makes it so you call the full_name(self) method like it is just a normal property, when in reality it is actually a method that contains code to be run when the property is set.



To predict on test data we need to traverse through each row of test data and each node of a tree and as well store the probabilities

In [161]:
def predict(self, data):
    return np.array([self.__flow_data_thru_tree(row) for _, row in data.iterrows()])
  
def __flow_data_thru_tree(self, row):
    if self.is_leaf_node: return self.probability
    tree = self.left if row[self.split_feature] <= self.criteria else self.right
    return tree.__flow_data_thru_tree(row)

#### Combining it all together

In [233]:
class DecisionTree:
    
    def __init__(self, max_depth = 6, depth = 1):
        self.left = None
        self.right = None
        self.max_depth = max_depth
        self.depth = depth

    def fit(self, data, target):
        if self.depth <= self.max_depth: print(f"processing at Depth: {self.depth}")
        self.data = data
        self.target = target
        self.independent = self.data.columns.tolist()
        self.independent.remove(target)
        if self.depth <= self.max_depth:
            
            self.__validate_data()
            self.impurity_score = self.__calculate_impurity_score(self.data[self.target])
            self.criteria, self.split_feature, self.information_gain = self.__find_best_split()
            if self.criteria is not None and self.information_gain > 0: 
                self.__create_branches()
        else: 
            print("Stopping splitting as Max depth reached")
            
        
    def __calculate_impurity_score(self, data):
        if data is None or data.empty: return 0
        probs = np.array(data.value_counts(normalize=True))
        return probs.dot(1-probs)
    
    def __find_best_split(self):
        best_split = {}
        for col in self.independent:
            information_gain, split = self.__find_best_split_for_column(col)
            if split is None: continue
            if not best_split or best_split["information_gain"] < information_gain:
                best_split = {"split": split, "col": col, "information_gain": information_gain}

        return best_split.get("split"), best_split.get("col"), best_split.get("information_gain")
    
    def __find_best_split_for_column(self, col):
        x = self.data[col]
        unique_values = x.unique()
        if len(unique_values) == 1: return None, None
        information_gain = None
        split = None
        # Iterate through all possible values
        for val in unique_values:
            left = x <= val
            right = x > val
            left_data = self.data[left]
            right_data = self.data[right]
            left_impurity = self.__calculate_impurity_score(left_data[self.target])
            right_impurity = self.__calculate_impurity_score(right_data[self.target])
            score = self.__calculate_information_gain(left_count = len(left_data),
                                                      left_impurity = left_impurity,
                                                      right_count = len(right_data),
                                                      right_impurity = right_impurity)
            # Store the best split
            if information_gain is None or score > information_gain: 
                information_gain = score 
                split = val
        return information_gain, split
    
    def __create_branches(self):
        self.left = DecisionTree(max_depth = self.max_depth, 
                                 depth = self.depth + 1)
        self.right = DecisionTree(max_depth = self.max_depth, 
                                 depth = self.depth + 1)
        left_rows = self.data[self.data[self.split_feature] <= self.criteria] 
        right_rows = self.data[self.data[self.split_feature] > self.criteria] 
        self.left.fit(data = left_rows, target = self.target)
        self.right.fit(data = right_rows, target = self.target) 
        
    def __calculate_information_gain(self, left_count, left_impurity, right_count, right_impurity):
        # weighted impurity score on nodes
        return self.impurity_score - ((left_count/len(self.data)) * left_impurity + \
                                      (right_count/len(self.data)) * right_impurity)
        
    def predict(self, data):
        return np.array([self.__flow_data_thru_tree(row) for row in data.values])

    def __flow_data_thru_tree(self, row):
        return self.data[self.target].value_counts() \
                   .apply(lambda x: x/len(self.data)).tolist()


    
        
    def __validate_data(self):
        non_numeric_columns = self.data[self.independent].select_dtypes(include=['category', 'object', 'bool']).columns.tolist()
        if(len(set(self.independent).intersection(set(non_numeric_columns))) != 0):
            raise RuntimeError("Not all columns are numeric")
        
        self.data[self.target] = self.data[self.target].astype("category")
#         if(len(self.data[self.target].cat.categories) != 2):
#             raise RuntimeError("Implementation is only for Binary Classification")
            
    @property
    def is_leaf_node(self): return self.left is None


    @property
    def probability(self):     
        return self.data[self.target].value_counts(normalize=True).to_dict()
    
    @property
    def class_predictions(self):     
        return self.data[self.target].value_counts(normalize=True).keys()[0]
    
    def predict_class(self, data):
        return np.array([self.__flow_data_thru_tree_hard_pred(row) for _, row in data.iterrows()])
    
    def predict_proba(self,data):
        return np.array([self.__flow_data_thru_tree(row) for _, row in data.iterrows()])
  
    def __flow_data_thru_tree(self, row):
        if self.is_leaf_node: return self.probability
        tree = self.left if row[self.split_feature] <= self.criteria else self.right
        return tree.__flow_data_thru_tree(row)
    
    def __flow_data_thru_tree_hard_pred(self, row):
        if self.is_leaf_node: return self.class_predictions
        tree = self.left if row[self.split_feature] <= self.criteria else self.right
        return tree.__flow_data_thru_tree_hard_pred(row)

    

#####  Test binary

In [222]:
train.Survived.value_counts(normalize=True).to_dict()

{0.0: 0.6161616161616161, 1.0: 0.3838383838383838}

In [223]:
model = DecisionTree(max_depth=3)
model.fit(train, 'Survived')

processing at Depth: 1
processing at Depth: 2
processing at Depth: 3
Stopping splitting as Max depth reached
Stopping splitting as Max depth reached
processing at Depth: 3
Stopping splitting as Max depth reached
Stopping splitting as Max depth reached
processing at Depth: 2
processing at Depth: 3
Stopping splitting as Max depth reached
Stopping splitting as Max depth reached
processing at Depth: 3
Stopping splitting as Max depth reached
Stopping splitting as Max depth reached


In [225]:
model.predict_proba(test)

array([{0.0: 0.9020618556701031, 1.0: 0.0979381443298969},
       {1.0: 0.64, 0.0: 0.36},
       {0.0: 0.9020618556701031, 1.0: 0.0979381443298969},
       {0.0: 0.9020618556701031, 1.0: 0.0979381443298969},
       {1.0: 0.64, 0.0: 0.36},
       {0.0: 0.9020618556701031, 1.0: 0.0979381443298969},
       {1.0: 0.64, 0.0: 0.36},
       {0.0: 0.7207207207207207, 1.0: 0.27927927927927926},
       {1.0: 0.64, 0.0: 0.36},
       {0.0: 0.9020618556701031, 1.0: 0.0979381443298969},
       {0.0: 0.9020618556701031, 1.0: 0.0979381443298969},
       {0.0: 0.9020618556701031, 1.0: 0.0979381443298969},
       {1.0: 0.8959537572254336, 0.0: 0.10404624277456648},
       {0.0: 0.9020618556701031, 1.0: 0.0979381443298969},
       {1.0: 0.8959537572254336, 0.0: 0.10404624277456648},
       {1.0: 0.8959537572254336, 0.0: 0.10404624277456648},
       {0.0: 0.9020618556701031, 1.0: 0.0979381443298969},
       {0.0: 0.9020618556701031, 1.0: 0.0979381443298969},
       {1.0: 0.64, 0.0: 0.36}, {1.0: 0.64, 0.0

In [226]:
model.predict_class(test)

array([0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0.,
       0., 1., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 1.,
       0., 0., 1., 1., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 1., 1., 0.,
       0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0.,
       0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0.,
       1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0.,
       1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 1., 1.,
       0., 0., 0., 0., 0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 1., 0., 1.,
       0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1.,
       0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1.,
       0., 1., 0., 1., 1.

##### Test multiclass

In [227]:
iris = load_iris()
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df['target'] = iris.target

In [228]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
sepal length (cm)    150 non-null float64
sepal width (cm)     150 non-null float64
petal length (cm)    150 non-null float64
petal width (cm)     150 non-null float64
target               150 non-null int64
dtypes: float64(4), int64(1)
memory usage: 5.9 KB


In [254]:
l = list(range(len(df)))
np.random.shuffle(l)
train_idx = l[0:int(len(l)*0.8)]
test_idx = l[int(len(l)*0.8):]

In [255]:
train = df.iloc[train_idx]
test = df.iloc[test_idx]

In [256]:
model = DecisionTree()
model.fit(train, 'target')

processing at Depth: 1
processing at Depth: 2
processing at Depth: 2
processing at Depth: 3
processing at Depth: 4
processing at Depth: 5
processing at Depth: 6
processing at Depth: 6
processing at Depth: 5
processing at Depth: 4
processing at Depth: 5
processing at Depth: 5
processing at Depth: 3
processing at Depth: 4
processing at Depth: 5
processing at Depth: 5
processing at Depth: 4


In [257]:
model.predict_class(test)

array([1, 0, 1, 2, 2, 2, 2, 1, 0, 0, 0, 0, 0, 2, 1, 0, 2, 0, 0, 1, 0, 2,
       2, 0, 1, 1, 0, 2, 1, 1])

In [258]:
np.sum(np.array(test.target) == model.predict_class(test))/ len(test)

0.9666666666666667